In [1181]:
from enum import Enum
from fractions import Fraction
from math import lcm

F = lambda a,b=1: Fraction(a).limit_denominator() / Fraction(b).limit_denominator()
LCD = lambda *f: lcm(*[frac.denominator for frac in f])

ROUND_PRECISION = 30
PRECISION_DISPLAY = 2

# Classes

### Ceilings

In [1182]:
class Ceiling:
    # eligible for RADD
    ONE_SISTER = F(1,2)
    TWO_SISTERS = F(2,3)
    BROTHER = F(1,1)
    MOTHER = F(1,2)
    
    # ?
    THREE_OR_MORE_SISTERS = F(1,1)
    
    # not eligible for RADD
    SON = F(1,1)
    NON_SOLE_DAUGHTER = F(1,1)
    FATHER = F(1,1)
    SOLE_DAUGHTER = F(1,2)

## Heirs

In [1183]:
class Heirs:
    def __init__(self, husband:bool=False, wife:bool=False, son:int=0, daughter:int=0, brother:int=0, sister:int=0, father:bool=False, mother:bool=False, relatives:int=0):
        self.husband = husband
        self.wife = wife
        self.son = son
        self.daughter = daughter
        self.brother = brother
        self.sister = sister
        self.father = father
        self.mother = mother
        self.relatives = relatives
        assert not (husband and wife), "Husband and wife cannot co-exist"
    
    @property
    def siblings(self):
        return self.brother + self.sister
    
    def is_multi_siblings(self):
        # TODO: i think (siblings > 2) is wrong, and should be (brother > 0 and sister > 0)
        # TODO: Also why > 1 and not > 0?
        return self.sister > 1 or self.brother > 1 or (self.siblings > 2)
    
    @property
    def children(self):
        return self.son + self.daughter
    
    @property
    def spouse(self) -> bool:
        return self.husband or self.wife
    
    @property
    def adults(self):
        return self.husband + self.wife + self.father + self.mother
    
    @property
    def parents(self) -> bool:
        return self.father or self.mother

    def __repr__(self):
        return f"husband: {self.husband}, wife: {self.wife}, son: {self.son}, daughter: {self.daughter}, brother: {self.brother}, sister: {self.sister}, father: {self.father}, mother: {self.mother}, relatives: {self.relatives}"

## Shares

In [1184]:
class Shares:
    def __init__(self):
        self.husband = 0
        self.wife = 0
        self.son = 0
        self.daughter = 0
        self.brother = 0
        self.sister = 0
        self.father = 0
        self.mother = 0
        self.relatives = 0
        self.total_radd = 0
        
    @property
    def spouse(self):
        return self.husband + self.wife

    def get_dict(self):
        return {"husband": self.husband, "wife": self.wife, "son": self.son, "daughter": self.daughter, "brother": self.brother, "sister": self.sister, "father": self.father, "mother": self.mother, "relatives": self.relatives}

    def total(self, with_radd=True):
        return self.husband + self.wife + self.son + self.daughter + self.brother + self.sister + self.father + self.mother + self.relatives + (self.total_radd if with_radd else 0)

    def remaining(self, with_radd=True):
        return 1 - self.total(with_radd=with_radd)

    def __repr__(self):
        return f"husband: {self.husband}, wife: {self.wife}, sons: {self.son}, daughter: {self.daughter}, brothers: {self.brother}, sisters: {self.sister}, father: {self.father}, mother: {self.mother}, relatives: {self.relatives}, Radd: {self.total_radd}"

## Amounts

In [1185]:
class Amounts:
    def __init__(self, heirs: Heirs, estate=24, amounts=None):
        self._heirs = heirs
        self._shares, self._radd = calculate_shares(heirs, estate=estate)
        self._estate = F(estate)
        if amounts is None:
            self._amounts = shares_to_amounts(self._shares, estate=estate)
        else:
            # TODO: For unittest
            self._amounts = amounts
            
    @property
    def estate(self):
        return self._estate
    
    def sum_amounts(self, with_radd=True):
        amounts = self._amounts
        return round(float(amounts.husband + amounts.wife + amounts.son + amounts.daughter + amounts.brother + amounts.sister + amounts.father + amounts.mother + amounts.relatives + (self._radd.total_radd if with_radd else 0)), ROUND_PRECISION)
    
    def remaining(self, with_radd=True):
        return self.estate - self.sum_amounts(with_radd=with_radd) - (self._radd.total() if with_radd else 0)
    
    def sum_shares(self):
        return self._shares.total()
    
    def one_heir_f_amount(self, heir_name):
        heirs_num = getattr(self._heirs, heir_name)
        return self.get_float_amount((getattr(self._amounts, heir_name) / heirs_num) if heirs_num > 0 else 0)
    
    def get_fraction(self, amount):
        try:
            return F(amount, self.estate)
        except TypeError:
            raise TypeError(f"F({amount},{self.estate})")
        # rounded_estate = round(self.estate)
        # rounded_amount = round(amount)
        # return F(rounded_amount, rounded_estate) if rounded_estate > 0 else 0
    
    def get_float_amount(self, amount):
        return float(amount)
        
    def __repr__(self):
        amounts = self._amounts
        heirs = self._heirs
        one_heir_amount = self.one_heir_f_amount
        get_fraction = self.get_fraction
        f_amount = self.get_float_amount
        return f"husband ({get_fraction(amounts.husband)}): {f_amount(amounts.husband):.{PRECISION_DISPLAY}f} (+{float(self._radd.husband):.{PRECISION_DISPLAY}f}), wife ({get_fraction(amounts.wife)}): {f_amount(amounts.wife):.{PRECISION_DISPLAY}f} (+{float(self._radd.wife):.{PRECISION_DISPLAY}f}), sons ({get_fraction(amounts.son)}): {heirs.son}x{one_heir_amount('son'):.{PRECISION_DISPLAY}f}, daughters ({get_fraction(amounts.daughter)}): {heirs.daughter}x{one_heir_amount('daughter'):.{PRECISION_DISPLAY}f} (+{float(self._radd.daughter):.{PRECISION_DISPLAY}f}), brothers ({get_fraction(amounts.brother)}): {heirs.brother}x{one_heir_amount('brother'):.{PRECISION_DISPLAY}f} (+{float(self._radd.brother):.{PRECISION_DISPLAY}f}), sisters ({get_fraction(amounts.sister)}): {heirs.sister}x{one_heir_amount('sister'):.{PRECISION_DISPLAY}f} (+{float(self._radd.sister):.{PRECISION_DISPLAY}f}), father ({get_fraction(amounts.father)}): {f_amount(amounts.father):.{PRECISION_DISPLAY}f}, mother ({get_fraction(amounts.mother)}): {f_amount(amounts.mother):.{PRECISION_DISPLAY}f} (+{float(self._radd.mother):.{PRECISION_DISPLAY}f}), relatives: {heirs.relatives}x{one_heir_amount('relatives'):.{PRECISION_DISPLAY}f}"

## Cases

In [1186]:
class Case(Enum):
    GENERAL_CASE = 'general case'
    ADULTS_CASE = 'adults case'
    MINOR_KALALA_CASE = 'minor kalala case'
    MAJOR_KALALA_CASE = 'major kalala case'

### get_case()

In [1187]:
def get_case(heirs: Heirs):
    if heirs.adults + heirs.children == 0:
        return Case.MAJOR_KALALA_CASE
    if (heirs.son > 0) or (heirs.daughter > 0 and heirs.adults > 0):
        return Case.GENERAL_CASE
    if (heirs.adults == 0) and (heirs.daughter > 0):
        return Case.MINOR_KALALA_CASE
    if (heirs.adults > 0) and (heirs.children == 0):
        return Case.ADULTS_CASE
    return Case.MAJOR_KALALA_CASE

# Examples

In [1188]:
general1 = Heirs(daughter=3, brother=2, sister=1, father=True)
assert get_case(general1) == Case.GENERAL_CASE, get_case(general1)

general2 = Heirs(daughter=3, brother=2, sister=1, mother=True, husband=True)
assert get_case(general2) == Case.GENERAL_CASE, get_case(general2)

general3 = Heirs(son=3, brother=2, sister=1, mother=True, husband=True)
assert get_case(general3) == Case.GENERAL_CASE, get_case(general3)

general4 = Heirs(son=2, daughter=1, brother=2, sister=1, mother=True, husband=True)
assert get_case(general4) == Case.GENERAL_CASE, get_case(general4)

general5 = Heirs(daughter=1, brother=2, sister=1, mother=True, husband=True)
assert get_case(general5) == Case.GENERAL_CASE, get_case(general5)

adults1 = Heirs(husband=True, father=True, brother=2, sister=1)
assert get_case(adults1) == Case.ADULTS_CASE, get_case(adults1)

adults2 = Heirs(husband=True, mother=True, brother=2, sister=1)
assert get_case(adults2) == Case.ADULTS_CASE, get_case(adults2)

adults3 = Heirs(wife=True, sister=1, brother=2)
assert get_case(adults3) == Case.ADULTS_CASE, get_case(adults3)

adults4 = Heirs(wife=True, sister=1, relatives=1)
assert get_case(adults4) == Case.ADULTS_CASE, get_case(adults4)

adults5 = Heirs(wife=True, sister=1, brother=1, relatives=1)
assert get_case(adults5) == Case.ADULTS_CASE, get_case(adults5)

minor_kalala_1 = Heirs(daughter=1)
assert get_case(minor_kalala_1) == Case.MINOR_KALALA_CASE, get_case(minor_kalala_1)

minor_kalala_2 = Heirs(daughter=3, brother=2, sister=1)
assert get_case(minor_kalala_2) == Case.MINOR_KALALA_CASE, get_case(minor_kalala_2)

minor_kalala_3 = Heirs(daughter=3, relatives=1)
assert get_case(minor_kalala_3) == Case.MINOR_KALALA_CASE, get_case(minor_kalala_3)

minor_kalala_4 = Heirs(daughter=1, relatives=1)
assert get_case(minor_kalala_4) == Case.MINOR_KALALA_CASE, get_case(minor_kalala_4)

major_kalala_1 = Heirs(brother=1)
assert get_case(major_kalala_1) == Case.MAJOR_KALALA_CASE, get_case(major_kalala_1)

major_kalala_2 = Heirs(brother=1, sister=1, relatives=1)
assert get_case(major_kalala_2) == Case.MAJOR_KALALA_CASE, get_case(major_kalala_2)

major_kalala_3 = Heirs(sister=1, relatives=1)
assert get_case(major_kalala_3) == Case.MAJOR_KALALA_CASE, get_case(major_kalala_3)

major_kalala_4 = Heirs(sister=2, relatives=1)
assert get_case(major_kalala_4) == Case.MAJOR_KALALA_CASE, get_case(major_kalala_4)

major_kalala_5 = Heirs(sister=3)
assert get_case(major_kalala_5) == Case.MAJOR_KALALA_CASE, get_case(major_kalala_5)

try:
    illegal = Heirs(husband=True, wife=True)
    assert False, f'assertion failed (should raise illegal): {illegal}'
except AssertionError as e:
    assert True

# Functions

## male_twice_as_female()

In [1189]:
def male_twice_as_female(estate, males, females):
    # M = 2F
    # M*m + F*f = ESTATE
    # 2F*m + F*f = ESTATE
    # F(2m + f) = ESTATE
    # F = ESTATE / (2m+f)
    # M = 2F
    one_female_share = estate / (2 * males + females)
    one_male_share = one_female_share * 2
    return one_male_share, one_female_share
    # TODO: can we use Fraction here?


## get_spouse_share()

In [1190]:
def get_spouse_share(heirs: Heirs, shares: Shares):
    if heirs.husband:  # wife died
        if heirs.children == 0:
            shares.husband = F(1,2)
        else:
            shares.husband = F(1,4)
    elif heirs.wife:  # husband died
        if heirs.children == 0:
            shares.wife = F(1,4)
        else:
            shares.wife = F(1,8)

## get_father_share()

In [1191]:
def get_father_share(heirs: Heirs, shares: Shares):
    if heirs.father:
        if heirs.children > 0:
            shares.father = F(1,6)
        # else rest
        # TODO: Check if we can simply calculate the rest and assign it here
        elif (heirs.mother and heirs.wife) or (not heirs.mother and heirs.husband):
            shares.father = F(1,2)
        elif heirs.mother and heirs.husband:
            shares.father = F(1,3)
        elif heirs.mother and not heirs.spouse:
            shares.father = F(2,3)
        elif not heirs.mother and heirs.wife:
            shares.father = F(3,4)
        elif not heirs.mother and not heirs.spouse:
            shares.father = 1
        else:
            shares.father = 0

## get_mother_share()

In [1192]:
def get_mother_share(heirs: Heirs, shares: Shares):
    if heirs.mother:
        # is_multi_siblings = heirs.is_multi_siblings()
        # mother_radd = 0
        if heirs.children:
            shares.mother = F(1,6)
        elif heirs.father:
            shares.mother = (1 - shares.spouse) * (F(1,3))
        elif heirs.siblings == 0:
            shares.mother = F(1,3)
            # RADD 1:
            # if heirs.husband or (heirs.wife and heirs.relatives) or (not heirs.spouse and heirs.relatives):
            #     shares.mother += F(1,6)
            # elif heirs.wife and not heirs.relatives:
            #     shares.mother += F(10,24) # TODO: ??
            # elif not heirs.spouse and not heirs.relatives:
            #     mother_radd += F(2,3)
        else:
            shares.mother = F(1,6)
            # RADD 2
            # if (heirs.husband and heirs.siblings < 2) or (heirs.wife and heirs.siblings >= 2):
            #     mother_radd += F(2,24)  # TODO ??
            # elif heirs.siblings == 1 and not heirs.spouse:
            #     mother_radd += F(1,3)
            # elif (is_multi_siblings and not heirs.spouse) or (not is_multi_siblings and heirs.brother == 1 and heirs.sister == 1 and not heirs.spouse):
            #     mother_radd += F(1,6)
        # heirs.mother += mother_radd

## get_children_share()

In [1193]:
def get_children_share(heirs: Heirs, shares: Shares):
    rest = shares.remaining()
    case = get_case(heirs)
    
    if heirs.children > 0:
        if case == Case.GENERAL_CASE:
            if heirs.daughter == 0:  # male only
                shares.son = rest
            elif heirs.son == 0:  # female only
                shares.daughter = rest
            else:  # mixed
                one_son_share, one_daughter_share = male_twice_as_female(rest, heirs.son, heirs.daughter)
                shares.son = one_son_share * heirs.son
                shares.daughter = one_daughter_share * heirs.daughter
        elif case == Case.MINOR_KALALA_CASE:
            if heirs.son == 0 and not heirs.spouse and not heirs.parents:
                if heirs.daughter == 1:
                    shares.daughter = F(1,2)
                elif heirs.daughter >= 2:
                    shares.daughter = F(2,3)

## get_children_radd()

In [1194]:
def get_children_radd_deprecated(heirs: Heirs, shares: Shares):
    # TODO: Why Radd for children?? I thought only mother and siblings get Radd??
    if get_case(heirs) == Case.MINOR_KALALA_CASE and heirs.children > 0:
        if heirs.son == 0 and not heirs.parents and not heirs.spouse and heirs.siblings and heirs.relatives == 0:
            if heirs.daughter == 1:
                shares.daughter = F(1,2)
            elif heirs.daughter > 1:
                shares.daughter = F(2,3)

## get_spouse_radd()

In [1195]:
def get_spouse_radd_deprecated(heirs: Heirs, shares: Shares):
    # TODO: Why Radd for husband / wife?? I thought only mother and siblings get Radd??
    
    # husband radd
    if heirs.husband:
        if heirs.siblings + heirs.relatives + heirs.children == 0:
            shares.husband += shares.remaining()
    
    # wife radd
    elif heirs.wife:
        if not heirs.parents and (heirs.children + heirs.siblings + heirs.relatives == 0):
            shares.wife += shares.remaining()

## get_siblings_share()

In [1196]:
def get_siblings_share(heirs: Heirs, shares: Shares):
    case = get_case(heirs)
    if case in [Case.ADULTS_CASE, Case.GENERAL_CASE]:
        # TODO: Looks like no need for general case, and minor and adults cases are the same
        if heirs.siblings > 0:
            if heirs.brother == 1 and heirs.siblings < 3 and case == Case.ADULTS_CASE and not heirs.father and heirs.son == 0 and (heirs.mother or heirs.spouse):
                shares.brother = F(1,6)
            if heirs.sister == 1 and heirs.siblings < 3 and case == Case.ADULTS_CASE and not heirs.father and heirs.son == 0 and (heirs.mother or heirs.spouse):
                shares.sister = F(1,6)
            if heirs.is_multi_siblings() and not heirs.father and not heirs.children:
                one_siblings_share = F(1,3) / heirs.siblings
                shares.brother = one_siblings_share * heirs.brother
                shares.sister = one_siblings_share * heirs.sister
    elif case == Case.MINOR_KALALA_CASE:
        if not heirs.father:
            if heirs.siblings < 3:
                if heirs.brother == 1:
                    shares.brother = F(1,6)
                elif heirs.sister == 1:
                    shares.sister = F(1,6)
            if heirs.brother > 1 or heirs.sister > 1 or heirs.siblings > 2:
                one_siblings_share = F(1,3) / heirs.siblings
                shares.brother = one_siblings_share * heirs.brother
                shares.sister = one_siblings_share * heirs.sister
    elif case == Case.MAJOR_KALALA_CASE:
        if heirs.brother == 0 and heirs.children == 0 and heirs.sister in [1,2]:
            if heirs.sister == 1: 
                shares.sister = F(1,2)
            elif heirs.sister == 2:
                shares.sister = F(2,3)
        elif heirs.sister == 0 and heirs.brother == 1 and heirs.children == 0:
                shares.brother = 1
        elif heirs.brother > 1 or heirs.sister > 2 or (heirs.brother > 0 and heirs.sister > 0):
            one_brother_share, one_sister_share = male_twice_as_female(1, heirs.brother, heirs.sister)
            shares.brother = one_brother_share * heirs.brother
            shares.sister = one_sister_share * heirs.sister

## get_siblings_radd()

In [1197]:
def get_siblings_radd_deprecated(heirs: Heirs, shares: Shares):
    # TODO: What's the data in T15,T16,T17? Corresponding U cells depend on them but ignore their values 
    if heirs.siblings:
        brother_radd = 0
        sister_radd = 0
        case = get_case(heirs)
        if case == Case.ADULTS_CASE:
            # brother radd
            if heirs.brother == 1 and heirs.siblings < 3 and not heirs.father and not heirs.son and (heirs.mother or heirs.parents):
                if (not heirs.sister and heirs.husband and not heirs.mother) or (not heirs.sister and heirs.mother and not heirs.spouse):
                    brother_radd = F(8,24)
                elif (not heirs.sister and heirs.husband and heirs.mother) or (heirs.wife and heirs.mother and heirs.sister == 1) or (heirs.husband == 1 and heirs.sister ==1 and not heirs.mother):
                    brother_radd = F(2,24)  # TODO ??
                elif (not heirs.sister and heirs.wife and heirs.mother) or (heirs.wife and not heirs.mother and heirs.sister == 1):
                    brother_radd = F(5,24)  # TODO ??
                elif not heirs.sister and heirs.wife and not heirs.mother:
                    brother_radd = F(14,24)  # TODO: ??
                elif heirs.mother and heirs.sister == 1 and not heirs.spouse:
                    brother_radd = F(4,24)
            # sister radd
            if heirs.sister == 1 and heirs.siblings < 3 and not heirs.father and not heirs.son and (heirs.mother or heirs.parents):
                if (not heirs.brother and heirs.husband and not heirs.mother) or (not heirs.brother and heirs.mother and not heirs.spouse):
                    sister_radd = F(8,24)
                elif (not heirs.brother and heirs.husband and heirs.mother) or (heirs.wife and heirs.mother and heirs.brother == 1) or (heirs.husband == 1 and heirs.brother ==1 and not heirs.mother):
                    sister_radd = F(2,24)  # TODO ??
                elif (not heirs.brother and heirs.wife and heirs.mother) or (heirs.wife and not heirs.mother and heirs.brother == 1):
                    sister_radd = F(5,24)  # TODO ??
                elif not heirs.brother and heirs.wife and not heirs.mother and heirs.relatives:  # TODO: why relatives important?
                    sister_radd = F(8,24)
                elif not heirs.brother and heirs.wife and not heirs.mother and not heirs.relatives:
                    sister_radd = F(14,24)  # TODO ??
                elif heirs.mother and heirs.brother == 1 and not heirs.spouse:
                    sister_radd = F(4,24)
            # several mixed case
            if heirs.is_multi_siblings() and not heirs.father and not heirs.children:
                if (heirs.husband and not heirs.mother and not heirs.wife) or (heirs.mother and not heirs.husband and heirs.wife):
                    one_sibling_radd = F(4,24) / heirs.siblings  # TODO ??
                    brother_radd = one_sibling_radd
                    sister_radd = one_sibling_radd
                elif (not heirs.mother and not heirs.husband and heirs.wife and (heirs.brother or heirs.sister > 2)) or (not heirs.mother and heirs.sister == 2 and not heirs.relatives):
                    one_sibling_radd = F(10,24) / heirs.siblings  # TODO ??
                    brother_radd = one_sibling_radd
                    sister_radd = one_sibling_radd
                elif (not heirs.spouse and heirs.mother) or (not heirs.mother and not heirs.husband and heirs.wife and heirs.sister == 2 and heirs.relatives and not heirs.brother):
                    one_sibling_radd = F(8,24) / heirs.siblings
                    brother_radd = one_sibling_radd
                    sister_radd = one_sibling_radd
        elif case == Case.MINOR_KALALA_CASE:
            W27 = heirs.brother == 1 and heirs.siblings < 3
            V27 = W27 and not heirs.father
            W28 = heirs.sister == 1 and heirs.siblings < 3
            V28 = W28 and not heirs.father
            W29 = heirs.is_multi_siblings()
            V29 = W29 and not heirs.father
            
            # brother radd
            if V27 and heirs.daughter < 2:
                if not V28:
                    brother_radd = F(8,24)
                else:
                    brother_radd = F(2,24)  # TODO ??
            elif V27 and heirs.daughter > 1 and not V28:
                brother_radd = F(4,24)
            
            # sister radd
            if V28 and heirs.daughter < 2:
                if not V27:
                    sister_radd = F(8,24)
                else:
                    sister_radd = F(2,24)  # TODO ??
            elif V28 and heirs.daughter > 1 and not V27:
                sister_radd = F(4,24)
            
            # mixed
            if V29 and heirs.daughter < 2:
                one_sibling_radd = F(4,24) / heirs.siblings
                brother_radd = one_sibling_radd
                sister_radd = one_sibling_radd
                
        elif case == Case.MAJOR_KALALA_CASE:
            # one sister
            W33 = heirs.sister == 1 and heirs.siblings < 2 and not heirs.children
            V33 = W33
            U33 = V33 and not heirs.spouse and not heirs.parents and not heirs.children and not heirs.brother and not heirs.relatives
            if U33:
                sister_radd = F(12,24)
            
            # two sisters
            W34 = heirs.sister == 2 and heirs.siblings < 3 and not heirs.children
            V34 = W34
            U34 = V34 and not heirs.spouse and not heirs.parents and not heirs.children and not heirs.brother and not heirs.relatives
            if U34:
                sister_radd = F(8,24)  # shared between 2 sisters
        
        shares.brother += brother_radd
        shares.sister += sister_radd

## get_relatives_share()

In [1198]:
def get_relatives_share(heirs: Heirs, shares: Shares):
    if heirs.relatives > 0 and shares.remaining() > 0:
        shares.relatives = shares.remaining()

## get_shares_general_case()

In [1199]:
def get_shares_general_case(heirs: Heirs):
    shares = Shares()

    # Husband / Wife
    get_spouse_share(heirs, shares)

    # Father
    get_father_share(heirs, shares)

    # Mother
    get_mother_share(heirs, shares)

    # Children
    get_children_share(heirs, shares)

    # Husband / Wife Radd
    # get_spouse_radd(heirs, shares)

    # Siblings
    get_siblings_share(heirs, shares)
    
    dummy_radd = Shares()

    # Relatives
    # TODO: i don't think relatives could inherit in general case
    get_relatives_share(heirs, shares)

    return shares, dummy_radd

## get_shares_adults_case()

In [1200]:
def get_shares_adults_case(heirs: Heirs, estate):
    shares = Shares()
    
    # Husband / Wife
    get_spouse_share(heirs, shares)
    
    # Father
    get_father_share(heirs, shares)
    
    # Mother
    get_mother_share(heirs, shares)
    
    # Husband / Wife Radd
    # get_spouse_radd(heirs, shares)
    
    # Siblings
    get_siblings_share(heirs, shares)
    
    # Radd
    radd = get_radd(heirs, shares, estate)

    # Relatives
    get_relatives_share(heirs, shares)
    
    # One heir
    transfer_rest(heirs, shares, radd, estate)
    
    return shares, radd

## get_shares_minor_kalala()

In [1201]:
def get_shares_minor_kalala(heirs: Heirs, estate):
    shares = Shares()
    
    # Daughter
    get_children_share(heirs, shares)
    
    # Siblings
    get_siblings_share(heirs, shares)

    # Radd
    radd = get_radd(heirs, shares, estate)
    
    # Relatives
    get_relatives_share(heirs, shares)
    
    # One heir
    transfer_rest(heirs, shares, radd, estate)
    
    return shares, radd

## get_shares_major_kalala()

In [1202]:
def get_shares_major_kalala(heirs: Heirs):
    shares = Shares()
    
    # Siblings
    get_siblings_share(heirs, shares)
    
    dummy_radd = Shares()
    
    # Relatives
    get_relatives_share(heirs, shares)
    
    return shares, dummy_radd

## RADD

### get_radd()

In [1237]:
def get_radd(heirs: Heirs, shares: Shares, estate) -> Shares:
    def _get_new_nominator(fraction : Fraction, new_denominator):
        return new_denominator / fraction.denominator * fraction.numerator
    
    radd = Shares()  # TODO: it's actually gonna be amounts, not shares... but Ok
    if get_case(heirs) in [Case.ADULTS_CASE, Case.MINOR_KALALA_CASE] and ((rem := (shares.remaining() * estate)) > 0):
        lcd = LCD(*shares.get_dict().values())
        
        # calc relative RADD
        # aX + bX + ... = remainder
        # X = remainder / sum(...)
        sister_new_numerator = _get_new_nominator(shares.sister,lcd)
        brother_new_numerator = _get_new_nominator(shares.brother,lcd)
        mother_new_numerator = _get_new_nominator(shares.mother,lcd)
        new_numerators_sum = sister_new_numerator + brother_new_numerator + mother_new_numerator
        relative_radd = F(rem, new_numerators_sum) if new_numerators_sum > 0 else 0
        
        # calc max radd portion
        sister_radd = relative_radd * F(sister_new_numerator)
        brothers_radd = relative_radd * F(brother_new_numerator)
        mother_radd = relative_radd * F(mother_new_numerator)
        
        # update ceilings
        if heirs.sister == 1:
            sisters_margin = Ceiling.ONE_SISTER - shares.sister
        elif heirs.sister == 2:
            sisters_margin = Ceiling.TWO_SISTERS - shares.sister
        else:
            # TODO: Probably no need for this case
            sisters_margin = Ceiling.THREE_OR_MORE_SISTERS - shares.sister
        
        brothers_margin = Ceiling.BROTHER - shares.brother
        mother_margin = Ceiling.MOTHER - shares.mother
        
        # calc actual radd shares
        radd.sister = min(F(sister_radd, estate), sisters_margin)
        radd.brother = min(F(brothers_radd, estate), brothers_margin)
        radd.mother = min(F(mother_radd, estate), mother_margin)
        
        # save radd
        shares.total_radd = radd.sister + radd.brother + radd.mother
        radd.total_radd = shares.total_radd * estate
        
        # calc radd amounts
        radd.sister = radd.sister * estate
        radd.brother = radd.brother * estate
        radd.mother = radd.mother * estate
        
    return radd

### transfer_rest()

In [1204]:
def transfer_rest(heirs: Heirs, shares: Shares, radd: Shares, estate):
    rest_portion = shares.remaining()
    if rest_portion > 0:
        rest_amount = rest_portion * estate
        if heirs.husband:
            radd.husband += rest_amount
        elif heirs.wife:
            radd.wife += rest_amount
        elif heirs.daughter > 0:  # minor kalala case
            radd.daughter += rest_amount
        
        radd.total_radd += rest_amount
        shares.total_radd += rest_portion

## shares_to_amounts()

In [1205]:
def shares_to_amounts(shares: Shares, estate=24) -> Shares:
    amounts = Shares()
    for heir_name, heir_share in shares.get_dict().items():
        setattr(amounts, heir_name, heir_share * estate)
    return amounts

## calculate_shares()

In [1206]:
def calculate_shares(heirs: Heirs, estate) -> (Shares,Shares):
    the_case = get_case(heirs)
    match the_case:
        case Case.GENERAL_CASE:
            return get_shares_general_case(heirs)
        case Case.ADULTS_CASE:
            return get_shares_adults_case(heirs, estate=estate)
        case Case.MINOR_KALALA_CASE:
            return get_shares_minor_kalala(heirs, estate=estate)
        case Case.MAJOR_KALALA_CASE:
            return get_shares_major_kalala(heirs)

# Calculations

## General

In [1207]:
general1_amounts = Amounts(general1)
print(f"{general1_amounts.sum_amounts()}/{general1_amounts.estate}")
print(f"{general1_amounts.sum_shares()}/1")
general1_amounts

24.0/24
1/1


husband (0): 0.00 (+0.00), wife (0): 0.00 (+0.00), sons (0): 0x0.00, daughters (5/6): 3x6.67 (+0.00), brothers (0): 2x0.00 (+0.00), sisters (0): 1x0.00 (+0.00), father (1/6): 4.00, mother (0): 0.00 (+0.00), relatives: 0x0.00

In [1208]:
general2_amounts = Amounts(general2)
print(f"{general2_amounts.sum_amounts()}/{general2_amounts.estate}")
print(f"{general2_amounts.sum_shares()}/1")
general2_amounts

24.0/24
1/1


husband (1/4): 6.00 (+0.00), wife (0): 0.00 (+0.00), sons (0): 0x0.00, daughters (7/12): 3x4.67 (+0.00), brothers (0): 2x0.00 (+0.00), sisters (0): 1x0.00 (+0.00), father (0): 0.00, mother (1/6): 4.00 (+0.00), relatives: 0x0.00

In [1209]:
general3_amounts = Amounts(general3)
print(f"{general3_amounts.sum_amounts()}/{general3_amounts.estate}")
print(f"{general3_amounts.sum_shares()}/1")
general3_amounts

24.0/24
1/1


husband (1/4): 6.00 (+0.00), wife (0): 0.00 (+0.00), sons (7/12): 3x4.67, daughters (0): 0x0.00 (+0.00), brothers (0): 2x0.00 (+0.00), sisters (0): 1x0.00 (+0.00), father (0): 0.00, mother (1/6): 4.00 (+0.00), relatives: 0x0.00

In [1210]:
general4_amounts = Amounts(general4)
print(f"{general4_amounts.sum_amounts()}/{general4_amounts.estate}")
print(f"{general4_amounts.sum_shares()}/1")
general4_amounts

24.0/24
1/1


husband (1/4): 6.00 (+0.00), wife (0): 0.00 (+0.00), sons (7/15): 2x5.60, daughters (7/60): 1x2.80 (+0.00), brothers (0): 2x0.00 (+0.00), sisters (0): 1x0.00 (+0.00), father (0): 0.00, mother (1/6): 4.00 (+0.00), relatives: 0x0.00

In [1211]:
general5_amounts = Amounts(general5)
print(f"{general5_amounts.sum_amounts()}/{general5_amounts.estate}")
print(f"{general5_amounts.sum_shares()}/1")
general5_amounts

24.0/24
1/1


husband (1/4): 6.00 (+0.00), wife (0): 0.00 (+0.00), sons (0): 0x0.00, daughters (7/12): 1x14.00 (+0.00), brothers (0): 2x0.00 (+0.00), sisters (0): 1x0.00 (+0.00), father (0): 0.00, mother (1/6): 4.00 (+0.00), relatives: 0x0.00

## Adults

In [1212]:
adults1_amounts = Amounts(adults1)
print(f"{adults1_amounts.sum_amounts()}/{adults1_amounts.estate}")
print(f"{adults1_amounts.sum_shares()}/1")
adults1_amounts

24.0/24
1/1


husband (1/2): 12.00 (+0.00), wife (0): 0.00 (+0.00), sons (0): 0x0.00, daughters (0): 0x0.00 (+0.00), brothers (0): 2x0.00 (+0.00), sisters (0): 1x0.00 (+0.00), father (1/2): 12.00, mother (0): 0.00 (+0.00), relatives: 0x0.00

In [1213]:
adults2_amounts = Amounts(adults2)
print(f"{adults2_amounts.sum_amounts()}/{adults2_amounts.estate}")
print(f"{adults2_amounts.sum_shares()}/1")
adults2_amounts

24.0/24
1/1


husband (1/2): 12.00 (+0.00), wife (0): 0.00 (+0.00), sons (0): 0x0.00, daughters (0): 0x0.00 (+0.00), brothers (2/9): 2x2.67 (+0.00), sisters (1/9): 1x2.67 (+0.00), father (0): 0.00, mother (1/6): 4.00 (+0.00), relatives: 0x0.00

In [1238]:
adults3_amounts = Amounts(adults3)
print(f"{adults3_amounts.sum_amounts()}/{adults3_amounts.estate}")
print(f"{adults3_amounts.sum_shares()}/1")
adults3_amounts

KeyboardInterrupt: 

In [1215]:
adults4_amounts = Amounts(adults4)
print(f"{adults4_amounts.sum_amounts()}/{adults4_amounts.estate}")
print(f"{adults4_amounts.sum_shares()}/1")
adults4_amounts

24.0/24
1/1


husband (0): 0.00 (+0.00), wife (1/4): 6.00 (+0.00), sons (0): 0x0.00, daughters (0): 0x0.00 (+0.00), brothers (0): 0x0.00 (+0.00), sisters (1/6): 1x4.00 (+8.00), father (0): 0.00, mother (0): 0.00 (+0.00), relatives: 1x6.00

In [1216]:
adults5_amounts = Amounts(adults5)
print(f"{adults5_amounts.sum_amounts()}/{adults5_amounts.estate}")
print(f"{adults5_amounts.sum_shares()}/1")
adults5_amounts

24.0/24
1/1


husband (0): 0.00 (+0.00), wife (1/4): 6.00 (+0.00), sons (0): 0x0.00, daughters (0): 0x0.00 (+0.00), brothers (1/6): 1x4.00 (+5.00), sisters (1/6): 1x4.00 (+5.00), father (0): 0.00, mother (0): 0.00 (+0.00), relatives: 1x0.00

## Minor Kalala

In [1217]:
minor_kalala_1_amounts = Amounts(minor_kalala_1)
print(f"{minor_kalala_1_amounts.sum_amounts()}/{minor_kalala_1_amounts.estate}")
print(f"{minor_kalala_1_amounts.sum_shares()}/1")
minor_kalala_1_amounts

24.0/24
1/1


husband (0): 0.00 (+0.00), wife (0): 0.00 (+0.00), sons (0): 0x0.00, daughters (1/2): 1x12.00 (+12.00), brothers (0): 0x0.00 (+0.00), sisters (0): 0x0.00 (+0.00), father (0): 0.00, mother (0): 0.00 (+0.00), relatives: 0x0.00

In [1218]:
minor_kalala_2_amounts = Amounts(minor_kalala_2)
print(f"{minor_kalala_2_amounts.sum_amounts()}/{minor_kalala_2_amounts.estate}")
print(f"{minor_kalala_2_amounts.sum_shares()}/1")
minor_kalala_2_amounts

24.0/24
1/1


husband (0): 0.00 (+0.00), wife (0): 0.00 (+0.00), sons (0): 0x0.00, daughters (2/3): 3x5.33 (+0.00), brothers (2/9): 2x2.67 (+0.00), sisters (1/9): 1x2.67 (+0.00), father (0): 0.00, mother (0): 0.00 (+0.00), relatives: 0x0.00

In [1219]:
minor_kalala_3_amounts = Amounts(minor_kalala_3)
print(f"{minor_kalala_3_amounts.sum_amounts()}/{minor_kalala_3_amounts.estate}")
print(f"{minor_kalala_3_amounts.sum_shares()}/1")
minor_kalala_3_amounts

24.0/24
1/1


husband (0): 0.00 (+0.00), wife (0): 0.00 (+0.00), sons (0): 0x0.00, daughters (2/3): 3x5.33 (+0.00), brothers (0): 0x0.00 (+0.00), sisters (0): 0x0.00 (+0.00), father (0): 0.00, mother (0): 0.00 (+0.00), relatives: 1x8.00

In [1220]:
minor_kalala_4_amounts = Amounts(minor_kalala_4)
print(f"{minor_kalala_4_amounts.sum_amounts()}/{minor_kalala_4_amounts.estate}")
print(f"{minor_kalala_4_amounts.sum_shares()}/1")
minor_kalala_4_amounts

24.0/24
1/1


husband (0): 0.00 (+0.00), wife (0): 0.00 (+0.00), sons (0): 0x0.00, daughters (1/2): 1x12.00 (+0.00), brothers (0): 0x0.00 (+0.00), sisters (0): 0x0.00 (+0.00), father (0): 0.00, mother (0): 0.00 (+0.00), relatives: 1x12.00

## Major Kalala

In [1221]:
major_kalala_1_amounts = Amounts(major_kalala_1)
print(f"{major_kalala_1_amounts.sum_amounts()}/{major_kalala_1_amounts.estate}")
print(f"{major_kalala_1_amounts.sum_shares()}/1")
major_kalala_1_amounts

24.0/24
1/1


husband (0): 0.00 (+0.00), wife (0): 0.00 (+0.00), sons (0): 0x0.00, daughters (0): 0x0.00 (+0.00), brothers (1): 1x24.00 (+0.00), sisters (0): 0x0.00 (+0.00), father (0): 0.00, mother (0): 0.00 (+0.00), relatives: 0x0.00

In [1222]:
major_kalala_2_amounts = Amounts(major_kalala_2)
print(f"{major_kalala_2_amounts.sum_amounts()}/{major_kalala_2_amounts.estate}")
print(f"{major_kalala_2_amounts.sum_shares()}/1")
major_kalala_2_amounts

24.0/24
1.0/1


husband (0): 0.00 (+0.00), wife (0): 0.00 (+0.00), sons (0): 0x0.00, daughters (0): 0x0.00 (+0.00), brothers (2/3): 1x16.00 (+0.00), sisters (1/3): 1x8.00 (+0.00), father (0): 0.00, mother (0): 0.00 (+0.00), relatives: 1x0.00

In [1223]:
major_kalala_3_amounts = Amounts(major_kalala_3)
print(f"{major_kalala_3_amounts.sum_amounts()}/{major_kalala_3_amounts.estate}")
print(f"{major_kalala_3_amounts.sum_shares()}/1")
major_kalala_3_amounts

24.0/24
1/1


husband (0): 0.00 (+0.00), wife (0): 0.00 (+0.00), sons (0): 0x0.00, daughters (0): 0x0.00 (+0.00), brothers (0): 0x0.00 (+0.00), sisters (1/2): 1x12.00 (+0.00), father (0): 0.00, mother (0): 0.00 (+0.00), relatives: 1x12.00

In [1224]:
major_kalala_4_amounts = Amounts(major_kalala_4)
print(f"{major_kalala_4_amounts.sum_amounts()}/{major_kalala_4_amounts.estate}")
print(f"{major_kalala_4_amounts.sum_shares()}/1")
major_kalala_4_amounts

24.0/24
1/1


husband (0): 0.00 (+0.00), wife (0): 0.00 (+0.00), sons (0): 0x0.00, daughters (0): 0x0.00 (+0.00), brothers (0): 0x0.00 (+0.00), sisters (2/3): 2x8.00 (+0.00), father (0): 0.00, mother (0): 0.00 (+0.00), relatives: 1x8.00

In [1225]:
major_kalala_5_amounts = Amounts(major_kalala_5)
print(f"{major_kalala_5_amounts.sum_amounts()}/{major_kalala_5_amounts.estate}")
print(f"{major_kalala_5_amounts.sum_shares()}/1")
major_kalala_5_amounts

24.0/24
1.0/1


husband (0): 0.00 (+0.00), wife (0): 0.00 (+0.00), sons (0): 0x0.00, daughters (0): 0x0.00 (+0.00), brothers (0): 0x0.00 (+0.00), sisters (1): 3x8.00 (+0.00), father (0): 0.00, mother (0): 0.00 (+0.00), relatives: 0x0.00

# Problems from episode 21

## print_results()

In [1226]:
def print_results(problem: Heirs):
    print(get_case(problem))
    problem_amounts = Amounts(problem)
    print(f"{problem_amounts.sum_amounts()}/{problem_amounts.estate}")
    print(f"{problem_amounts.sum_shares()}/1")
    print(problem_amounts)

## Problem 1

In [1227]:
p1 = Heirs(husband=True, mother=True)
print_results(p1)

Case.ADULTS_CASE
24.0/24
1/1
husband (1/2): 12.00 (+0.00), wife (0): 0.00 (+0.00), sons (0): 0x0.00, daughters (0): 0x0.00 (+0.00), brothers (0): 0x0.00 (+0.00), sisters (0): 0x0.00 (+0.00), father (0): 0.00, mother (1/3): 8.00 (+4.00), relatives: 0x0.00


## Problem 2

In [1228]:
p2 = Heirs(wife=True, mother=True, father=True, sister=3)
print_results(p2)

Case.ADULTS_CASE
24.0/24
1/1
husband (0): 0.00 (+0.00), wife (1/4): 6.00 (+0.00), sons (0): 0x0.00, daughters (0): 0x0.00 (+0.00), brothers (0): 0x0.00 (+0.00), sisters (0): 3x0.00 (+0.00), father (1/2): 12.00, mother (1/4): 6.00 (+0.00), relatives: 0x0.00


## Problem 3

In [1229]:
p3 = Heirs(husband=True, mother=True, sister=2, relatives=1)
print_results(p3)

Case.ADULTS_CASE
24.0/24
1/1
husband (1/2): 12.00 (+0.00), wife (0): 0.00 (+0.00), sons (0): 0x0.00, daughters (0): 0x0.00 (+0.00), brothers (0): 0x0.00 (+0.00), sisters (1/3): 2x4.00 (+0.00), father (0): 0.00, mother (1/6): 4.00 (+0.00), relatives: 1x0.00


## Problem 4

In [1230]:
p4 = Heirs(husband=True, father=True, mother=True, brother=3, son=1, daughter=1)
print_results(p4)

Case.GENERAL_CASE
24.0/24
1/1
husband (1/4): 6.00 (+0.00), wife (0): 0.00 (+0.00), sons (5/18): 1x6.67, daughters (5/36): 1x3.33 (+0.00), brothers (0): 3x0.00 (+0.00), sisters (0): 0x0.00 (+0.00), father (1/6): 4.00, mother (1/6): 4.00 (+0.00), relatives: 0x0.00


## Problem 5

In [1231]:
p5 = Heirs(husband=True, father=True, mother=True, daughter=2, brother=3, relatives=1)
print_results(p5)

Case.GENERAL_CASE
24.0/24
1/1
husband (1/4): 6.00 (+0.00), wife (0): 0.00 (+0.00), sons (0): 0x0.00, daughters (5/12): 2x5.00 (+0.00), brothers (0): 3x0.00 (+0.00), sisters (0): 0x0.00 (+0.00), father (1/6): 4.00, mother (1/6): 4.00 (+0.00), relatives: 1x0.00


## Problem 6

In [1232]:
p6 = Heirs(wife=True, daughter=1, sister=1, brother=5, relatives=5)
print_results(p6)

Case.GENERAL_CASE
24.0/24
1/1
husband (0): 0.00 (+0.00), wife (1/8): 3.00 (+0.00), sons (0): 0x0.00, daughters (7/8): 1x21.00 (+0.00), brothers (0): 5x0.00 (+0.00), sisters (0): 1x0.00 (+0.00), father (0): 0.00, mother (0): 0.00 (+0.00), relatives: 5x0.00


## Problem 7

In [1233]:
p7 = Heirs(daughter=1, sister=1)
print_results(p7)

Case.MINOR_KALALA_CASE
24.0/24
1/1
husband (0): 0.00 (+0.00), wife (0): 0.00 (+0.00), sons (0): 0x0.00, daughters (1/2): 1x12.00 (+0.00), brothers (0): 0x0.00 (+0.00), sisters (1/6): 1x4.00 (+8.00), father (0): 0.00, mother (0): 0.00 (+0.00), relatives: 0x0.00


## Problem 8

In [1234]:
p8 = Heirs(daughter=2, sister=5, brother=4)
print_results(p8)

Case.MINOR_KALALA_CASE
24.0/24
1/1
husband (0): 0.00 (+0.00), wife (0): 0.00 (+0.00), sons (0): 0x0.00, daughters (2/3): 2x8.00 (+0.00), brothers (4/27): 4x0.89 (+0.00), sisters (5/27): 5x0.89 (+0.00), father (0): 0.00, mother (0): 0.00 (+0.00), relatives: 0x0.00


## Problem 9

In [1235]:
p9 = Heirs(sister=2, relatives=1)
print_results(p9)

Case.MAJOR_KALALA_CASE
24.0/24
1/1
husband (0): 0.00 (+0.00), wife (0): 0.00 (+0.00), sons (0): 0x0.00, daughters (0): 0x0.00 (+0.00), brothers (0): 0x0.00 (+0.00), sisters (2/3): 2x8.00 (+0.00), father (0): 0.00, mother (0): 0.00 (+0.00), relatives: 1x8.00


## Problem 10

In [1236]:
p10 = Heirs(sister=3, relatives=1)
print_results(p10)

Case.MAJOR_KALALA_CASE
24.0/24
1.0/1
husband (0): 0.00 (+0.00), wife (0): 0.00 (+0.00), sons (0): 0x0.00, daughters (0): 0x0.00 (+0.00), brothers (0): 0x0.00 (+0.00), sisters (1): 3x8.00 (+0.00), father (0): 0.00, mother (0): 0.00 (+0.00), relatives: 1x0.00
